## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-20-08-47-27 +0000,bbc,UK signs deal with Iraq to speed up migrant re...,https://www.bbc.com/news/articles/c1w85yrer81o...
1,2025-08-20-08-46-01 +0000,bbc,Air fares and food prices push up inflation in...,https://www.bbc.com/news/articles/c741wkngndqo...
2,2025-08-20-08-44-20 +0000,bbc,Family of NZ fugitive on the run with children...,https://www.bbc.com/news/articles/c3v37ye0l0ro...
3,2025-08-20-08-31-53 +0000,bbc,Dozens of Afghan deportees from Iran killed in...,https://www.bbc.com/news/articles/c336jj5xkvko...
4,2025-08-20-08-31-30 +0000,nypost,Ex-Space Force sergeant Orest Schur sentenced ...,https://nypost.com/2025/08/20/us-news/ex-space...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2304/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
119,trump,48
339,ukraine,26
2,deal,16
396,russia,14
106,will,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
111,2025-08-19-22-54-43 +0000,nypost,Russia launches largest strike on Ukraine in w...,https://nypost.com/2025/08/19/world-news/russi...,127
275,2025-08-19-14-07-00 +0000,wsj,"After months of effort, President Trump says a...",https://www.wsj.com/world/trump-plans-to-get-p...,126
161,2025-08-19-20-30-23 +0000,latimes,Contributor: Trump's Russia and Ukraine summit...,https://www.latimes.com/opinion/story/2025-08-...,114
43,2025-08-20-04-23-23 +0000,bbc,Kremlin plays down Zelensky talks as Trump war...,https://www.bbc.com/news/articles/cn92e52rpjxo...,107
298,2025-08-19-10-53-00 +0000,wsj,Europe Defense Stocks Weaker Amid Trump’s Push...,https://www.wsj.com/world/europe/european-defe...,103


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
111,127,2025-08-19-22-54-43 +0000,nypost,Russia launches largest strike on Ukraine in w...,https://nypost.com/2025/08/19/world-news/russi...
297,56,2025-08-19-10-58-32 +0000,nyt,Why European Leaders Want a Cease-Fire in Ukra...,https://www.nytimes.com/2025/08/19/us/ukraine-...
31,47,2025-08-20-05-28-12 +0000,nypost,Ex-coach at top Iowa gymnastics academy arrest...,https://nypost.com/2025/08/20/us-news/ex-coach...
174,45,2025-08-19-19-55-47 +0000,nypost,Texas Dem Rep. Nicole Collier slept in state H...,https://nypost.com/2025/08/19/us-news/texas-de...
218,43,2025-08-19-18-03-18 +0000,latimes,SoCal's worst heat wave this year: How long wi...,https://www.latimes.com/california/story/2025-...
216,39,2025-08-19-18-16-27 +0000,nyt,Why Were Air Canada Flight Attendants Striking...,https://www.nytimes.com/2025/08/19/business/ai...
46,35,2025-08-20-04-11-09 +0000,nypost,Beloved TV hair and make-up professional Travi...,https://nypost.com/2025/08/20/us-news/beloved-...
261,34,2025-08-19-14-54-58 +0000,nyt,Netanyahu Faces Pressure From Far Right Over N...,https://www.nytimes.com/2025/08/19/world/middl...
234,33,2025-08-19-17-09-45 +0000,nypost,China’s Pop Mart sees profit soar 400% as Labu...,https://nypost.com/2025/08/19/business/chinas-...
117,33,2025-08-19-22-29-36 +0000,nypost,Socialist Zohran Mamdani holds private meeting...,https://nypost.com/2025/08/19/us-news/zohran-m...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
